<a href="https://colab.research.google.com/github/FabienMiguel/NLP-Fellowship/blob/main/Fabien_Mbonigaba_Hackathon1_File.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install fastapi pyngrok uvicorn nest-asyncio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.0 MB/s 
     |████████████████████████████████| 745 kB 38.5 MB/s 
     |████████████████████████████████| 56 kB 4.5 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 80 kB 9.5 MB/s 
     |████████████████████████████████| 58 kB 6.2 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=683e114e5dccd858f5a08a4a6a719de304084e5316b46d7e36dca2fee946e587
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [3]:
!pip install -U easynmt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 47.9 MB/s 
     |████████████████████████████████| 1.3 MB 52.7 MB/s 
     |████████████████████████████████| 68 kB 6.5 MB/s 
     |████████████████████████████████| 163 kB 69.9 MB/s 
     |████████████████████████████████| 7.6 MB 49.6 MB/s 
  Using cached pybind11-2.10.1-py3-none-any.whl (216 kB)
  Created wheel for easynmt: filename=EasyNMT-2.0.2-py3-none-any.whl size=19919 sha256=746ef0edfd3cd6a103c1cb4f402220d809664b29a2452764d7130c44d9e7e9e9
  Stored in directory: /root/.cache/pip/wheels/d1/57/06/53ca38645e14d4537a41e5a36da2026e10f54ae88240dd5190
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3155586 sha256=2a0c192cd5345bf09fe71047d0ca4ea1fff5fff61d481022b1377d19f0653fa4
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built easy

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from fastapi import FastAPI, Response
from pyngrok import ngrok
import nest_asyncio
import uvicorn
from starlette.responses import HTMLResponse
import json
from easynmt import EasyNMT

In [5]:
# set authentication for ngrok
!ngrok authtoken 2HJQ5KFUMPejTBPPFOQGP2WRU74_o2vYuu2ZQcGb3zYD6ziq

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [6]:
x = '2HJPvK5e4VxcduBjO5XSc0LqqCR_5zV3b5gBcKuNMUnxdpfuP'.split('_')

In [7]:
len(x[1])

21

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**draft**

In [9]:
# SOUP
link = 'https://www.jobinrwanda.com/jobs/featured'
soup = BeautifulSoup(requests.get(link).content, 'html.parser')

In [10]:
# LINKS OF ALL FEATURED CONTENT ON WEB
# job title
job_titles = {}

divs = soup.find_all('div', class_ = 'card-body p-2')
f_links = []

for index,div in enumerate(divs):
  anchor = div.find('a')
  found_link = 'https://www.jobinrwanda.com'+anchor['href']
  job_titles[index] = anchor.find('h5', class_ = 'card-title').get_text()
  f_links.append(found_link)
job_titles = list(job_titles.values())

In [11]:
job_titles

['Hiring a Firm for BRD Maintenance Assessment\n',
 'Construction du Guichet de Byumba\n',
 'Consultant- Monitoring & Evaluation\n',
 'Operations Manager\n',
 'Terms of Reference for Production Of the Second Generation of the Country Gender Status Report for Rwanda\n',
 'Call for Application for Professional Training on use of\xa0EBM, E-procurement and Tax Declaration through RRA\n',
 'Account Manager- Credit\n',
 'Conducting a Tripartite Assessment of Skills Leading Directly to Decent Employment\n',
 'Legal Manager\n',
 'Development of Debit Card Bridge (API)\n',
 "Itangazo rya Cyamunara y'Imodoka\n",
 'Expression of Interest (EoI) for New Construction/Building works at the Rwanda Institute for Conservation agriculture in Bugasera\n',
 'Expression of Interest (EOI) Event management for the Pharma Conference in Kigali, Rwanda\n',
 'Supply ICT Equipment\n',
 'Supply of Laptop Computers\xa0\n',
 'District Coordinator/Junior District Manager\n',
 'Recruitment of (a) Consultancy Firm to Pr

In [12]:
# EACH LINK SOUP
l_soups = {}
for index,link in enumerate(f_links):
  SOUP = BeautifulSoup(requests.get(link).content, 'html.parser')
  l_soups[index] = SOUP

In [13]:
# DIVS FOR EACH FEATURE CONTENT
content_divs = {}
job_info = {}
for index,soup1 in enumerate(list(l_soups.values())):
  content_divs[index] = soup1.find('div', class_ = 'clearfix text-formatted field field--name-field-job-full-description field--type-text-long field--label-hidden field__item')
  ul = soup1.find('ul', class_ = 'list-group list-group-flush')
  lists = {}
  for li in ul.children:
    try:
      info = re.sub('\s{2,}|\n',' ',li.text) # both key and value together as str
      # separate key & value to get key:value dictionary
      if not 'apply' in info.lower():
        if 'viewed' in info:
          times = re.search('\d+', info).group()
          lists['views'] = times
        else:
          k,value = re.split(':', info)[:2]
          lists[k.lstrip()] = value
    except AttributeError:
      pass
  job_info[index] = lists


In [14]:
# FULL CONTENT OF EACH FEATURE
full_content = {}
for index,div in enumerate(list(content_divs.values())):
  one_div_content = []
  for para in div.find_all('p', class_ = 'text-align-justify'):
    one_div_content.append(para.get_text())
  full_content[index] = re.sub('\n|\\xa0','','\n'.join(one_div_content))

In [15]:
#[['location: Kigali','views','job des','feature 1 content'],['feature 2'],[],[]]
final = []
for index in list(content_divs.keys()):
  index_info = list(job_info[index].values())
  index_text = [full_content[index]]
  final.append(index_info + index_text)
final


[['149',
  ' , RW ',
  ' Other ',
  ' Not specified',
  ' Not specified ',
  ' Contract ',
  ' Friday, 25/11/2022 10',
  ' 1',
  'RE-ADVERTISEMENT OFTENDER : Nº: 022/10/2022/BRD/HC&CS/CSASSIGNMENT TITLE:HIRING A FIRM FOR BRD MAINTENANCE ASSESSMENTThe Development Bank of Rwanda would like to hire a firm for BRD Maintenance Assessment of its different equipment infrastructure that support on daily basis the Bank’s operations.In this regard, the Development Bank of Rwanda (BRD) invites the interested and potential firms to submit their bids for the above-mentioned services according to the technical specification detailed in the Request for Proposal.The site visit is planned on 17thNovember 2022.The Request for Proposal in English may be obtained from procurement office from 10thNovember 2022 during office hours(8:00 am to 6:00 pm, Local time)upon presentation of proof of payment of a non-refundable fee ofTwenty Thousand (20,000 Frw)deposited on the accountNº 593412700910118 of BRD opened

In [16]:
columns = list(job_info[0].keys()) + ['Description']
len(columns)

9

In [17]:
dataframe = pd.DataFrame(final, columns = columns)

In [18]:
dataframe['job title'] = job_titles

In [19]:
dataframe['job title'] = dataframe['job title'].str.replace('\n','',regex=True).astype('str')

In [20]:
only_IT_jobs = dataframe.copy()
only_IT_jobs['is IT job'] = False

for index in range(len(only_IT_jobs)):
  if re.findall('Computer|IT|Software', only_IT_jobs.iloc[index]['Sector']):
    only_IT_jobs['is IT job'][index] = True

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [21]:
only_IT_jobs = only_IT_jobs[only_IT_jobs['is IT job'] == True]   # filter for the IT job

In [22]:
only_IT_jobs

,views,Location,Sector,Education level,Desired experience,Contract type,Deadline,Number of positions,Description,job title,is IT job
29,248,"Kigali , RW","Computer and IT, Management, Project management",Bachelor,Senior (5+ years of experience),Full-time,"Tuesday, 10/01/2023 23",1,Do you want to do work that matters? Do you wa...,Product Manager,True
120,1474,"Kigali , RW","Computer and IT, Other, Project management",Bachelor,Senior (5+ years of experience),Full-time,"Friday, 18/11/2022 23",1,CHANCEN International is a non-profit organiza...,IT and MIS Director,True
125,349,"Kigali , RW","Agriculture, Agronomy, Business, Computer and...",Bachelor,Mid career (3 to 5 years of experience),Full-time,"Wednesday, 25/01/2023 23",1,"About One Acre FundFounded in 2006, One Acre F...",Global Remote Sensing Manager,True
135,802,", RW","Computer and IT, Education, Other",Bachelor,Not specified,Full-time,"Monday, 28/11/2022 23",1,DescriptionJob Title:Instructional Technology ...,Instructional Technology Assistant,True
168,2742,"Kigali , RW","Agriculture, Business, Computer and IT, Manag...",Not specified,Not specified,Full-time,"Tuesday, 06/12/2022 23",1,"About One Acre FundFounded in 2006, One Acre F...",IT Operations Senior Manager,True


# umucyo webscrapping

In [23]:
url = 'https://www.umucyo.gov.rw/eb/bav/selectListAdvertisingListForGU.do'
umucyo_soup = BeautifulSoup(requests.get(url).content, 'html.parser')
table = umucyo_soup.find('table', class_ = 'article_table mb10')

In [24]:
head_row = table.find('thead').find('tr')
titles = []
for th in head_row.find_all('th'):
  titles.append(re.sub('\r|\n|\t','',th.text))
titles = titles[1:]

In [25]:
titles

['Tender Name',
 'Tender No',
 'Status',
 'Advertising Date',
 'Deadline of Submitting',
 'Planed Open Date',
 'Stage Type']

In [26]:
# table body
body_rows = table.find('tbody').find_all('tr') 
full_data = []
for row in body_rows:
  ls = []
  for detail in row.find_all('td'):
    ls.append(re.sub('\n|\r|\t|\s', '',detail.text))
  full_data.append(ls[1:])


In [27]:
full_data

[['HiringconsultantUrbanMobility(Publictransport)tosupportRwandaurbanmobilityproject',
  '000008/C/ICB/2022/2023/CoK',
  'Published',
  '10/11/2022',
  '27/12/202210:00',
  '27/12/202210:30',
  'onestage'],
 ['HiringconsultantTransporteconomisttosupportRwandaurbanMobilityProject',
  '000009/C/ICB/2022/2023/CoK',
  'Published',
  '10/11/2022',
  '27/12/202210:00',
  '27/12/202210:30',
  'onestage'],
 ['sciencekitsandlaboratoryequipment',
  '000001/G/ICB/2022/2023/REB',
  'Published',
  '03/11/2022',
  '20/12/202210:00',
  '20/12/202210:30',
  'onestage'],
 ['HiringaninternationalExpertinDataProcessingtosupport2022populationandHousingcensus.',
  '000008/C/ICB/2022/2023/NISR',
  'Published',
  '02/11/2022',
  '19/12/202210:00',
  '19/12/202210:15',
  'onestage'],
 ['TENDERFOROFFICESUPPLIESANDSTATIONARY',
  '000001/G/NCB/2022/2023/Kiziguro',
  'Published',
  '04/11/2022',
  '16/12/202210:00',
  '16/12/202210:30',
  'onestage'],
 ['supplyoflabokitsreagentsandconsumables',
  '000034/G/NCB/20

In [28]:
umucyo_df = pd.DataFrame(full_data, columns = titles)

In [29]:
umucyo_df.head(10)

,Tender Name,Tender No,Status,Advertising Date,Deadline of Submitting,Planed Open Date,Stage Type
0,HiringconsultantUrbanMobility(Publictransport)...,000008/C/ICB/2022/2023/CoK,Published,10/11/2022,27/12/202210:00,27/12/202210:30,onestage
1,HiringconsultantTransporteconomisttosupportRwa...,000009/C/ICB/2022/2023/CoK,Published,10/11/2022,27/12/202210:00,27/12/202210:30,onestage
2,sciencekitsandlaboratoryequipment,000001/G/ICB/2022/2023/REB,Published,03/11/2022,20/12/202210:00,20/12/202210:30,onestage
3,HiringaninternationalExpertinDataProcessingtos...,000008/C/ICB/2022/2023/NISR,Published,02/11/2022,19/12/202210:00,19/12/202210:15,onestage
4,TENDERFOROFFICESUPPLIESANDSTATIONARY,000001/G/NCB/2022/2023/Kiziguro,Published,04/11/2022,16/12/202210:00,16/12/202210:30,onestage
5,supplyoflabokitsreagentsandconsumables,000034/G/NCB/2022/2023/RAB,Published,10/11/2022,15/12/202211:00,15/12/202211:30,onestage
6,Provisionofservicesofraisingawarenessonstandar...,000010/NC/NCB/2022/2023/RSB,Published,11/11/2022,15/12/202210:00,15/12/202210:10,onestage
7,"Rehabilitationofadministrationbloc,constructio...",000001/W/NCB/2022/2023/NIDA,Published,09/11/2022,15/12/202210:00,15/12/202210:30,onestage
8,HiringaconsultantfirmforStudyreviewandsupervis...,000014/C/NCB/2022/2023/CoK,Published,11/11/2022,15/12/202209:00,15/12/202209:10,onestage
9,Supplyofcalibrationcertifiedreferencematerials...,000009/G/NCB/2022/2023/RSB,Published,11/11/2022,15/12/202209:00,15/12/202209:30,onestage


#dataframe translation

In [30]:
app = FastAPI(title = 'My hackathon api')
@app.get('/')
def home():
  return 'Welcome to Fabien Miguel Hackathon'

@app.get('/ ')
def index():
    
    return 'My name is Miguel, This is my first API'

In [31]:
tunnel = ngrok.connect(8000)
print('YOUR PUBLIC IP ADRRES: ', tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port = 8000)

INFO:     Started server process [79]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


YOUR PUBLIC IP ADRRES:  http://24ac-34-142-169-164.ngrok.io


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [79]


In [ ]:
model = EasyNMT('opus-mt')
for index in range(len(only_IT_jobs)):
  try:
    z = model.translate([only_IT_jobs.iloc[index]['Description']], target_lang = 'fr', source_lang = 'en')
    print('TRANSLATED:   ', z)
    only_IT_jobs['job title'][index] = model.translate([only_IT_jobs.iloc[index]['job title']], target_lang = 'fr', source_lang = 'en')
    only_IT_jobs['Description'][index] = model.translate([only_IT_jobs.iloc[index]['Description']], target_lang = 'fr', source_lang = 'en')
    if index != len(only_IT_jobs) - 1:
      print(f'''{index}:{len(only_IT_jobs)} translated''')
  except Exception as e:
    print(e)

In [ ]:
only_IT_jobs.head()

In [34]:
!pkill ngrok

In [35]:
app = FastAPI(title = 'MY API')
@app.get('/')
def start():
  return "this is the beginning of the page"

In [36]:
langs = '''{"Abkhaz": "ab",
 "Afar": "aa",
 "Afrikaans": "af",
 "Akan": "ak",
 "Albanian": "sq",
 "Amharic": "am",
 "Arabic": "ar",
 "Aragonese": "an",
 "Armenian": "hy",
 "Assamese": "as",
 "Avaric": "av",
 "Avestan": "ae",
 "Aymara": "ay",
 "Azerbaijani": "az",
 "Bambara": "bm",
 "Bashkir": "ba",
 "Basque": "eu",
 "Belarusian": "be",
 "Bengali": "bn",
 "Bihari": "bh",
 "Bislama": "bi",
 "Bosnian": "bs",
 "Breton": "br",
 "Bulgarian": "bg",
 "Burmese": "my",
 "Catalan; Valencian": "ca",
 "Chamorro": "ch",
 "Chechen": "ce",
 "Chichewa; Chewa; Nyanja": "ny",
 "Chinese": "zh",
 "Chuvash": "cv",
 "Cornish": "kw",
 "Corsican": "co",
 "Cree": "cr",
 "Croatian": "hr",
 "Czech": "cs",
 "Danish": "da",
 "Divehi; Dhivehi; Maldivian;": "dv",
 "Dutch": "nl",
 "English": "en",
 "Esperanto": "eo",
 "Estonian": "et",
 "Ewe": "ee",
 "Faroese": "fo",
 "Fijian": "fj",
 "Finnish": "fi",
 "French": "fr",
 "Fula; Fulah; Pulaar; Pular": "ff",
 "Galician": "gl",
 "Georgian": "ka",
 "German": "de",
 "Greek, Modern": "el",
 "Guaraní": "gn",
 "Gujarati": "gu",
 "Haitian; Haitian Creole": "ht",
 "Hausa": "ha",
 "Hebrew (modern)": "he",
 "Herero": "hz",
 "Hindi": "hi",
 "Hiri Motu": "ho",
 "Hungarian": "hu",
 "Interlingua": "ia",
 "Indonesian": "id",
 "Interlingue": "ie",
 "Irish": "ga",
 "Igbo": "ig",
 "Inupiaq": "ik",
 "Ido": "io",
 "Icelandic": "is",
 "Italian": "it",
 "Inuktitut": "iu",
 "Japanese": "ja",
 "Javanese": "jv",
 "Kalaallisut, Greenlandic": "kl",
 "Kannada": "kn",
 "Kanuri": "kr",
 "Kashmiri": "ks",
 "Kazakh": "kk",
 "Khmer": "km",
 "Kikuyu, Gikuyu": "ki",
 "Kinyarwanda": "rw",
 "Kirghiz, Kyrgyz": "ky",
 "Komi": "kv",
 "Kongo": "kg",
 "Korean": "ko",
 "Kurdish": "ku",
 "Kwanyama, Kuanyama": "kj",
 "Latin": "la",
 "Luxembourgish, Letzeburgesch": "lb",
 "Luganda": "lg",
 "Limburgish, Limburgan, Limburger": "li",
 "Lingala": "ln",
 "Lao": "lo",
 "Lithuanian": "lt",
 "Luba-Katanga": "lu",
 "Latvian": "lv",
 "Manx": "gv",
 "Macedonian": "mk",
 "Malagasy": "mg",
 "Malay": "ms",
 "Malayalam": "ml",
 "Maltese": "mt",
 "Māori": "mi",
 "Marathi (Marāṭhī)": "mr",
 "Marshallese": "mh",
 "Mongolian": "mn",
 "Nauru": "na",
 "Navajo, Navaho": "nv",
 "Norwegian Bokmål": "nb",
 "North Ndebele": "nd",
 "Nepali": "ne",
 "Ndonga": "ng",
 "Norwegian Nynorsk": "nn",
 "Norwegian": "no",
 "Nuosu": "ii",
 "South Ndebele": "nr",
 "Occitan": "oc",
 "Ojibwe, Ojibwa": "oj",
 "Old Church Slavonic, Church Slavic, Church Slavonic, Old Bulgarian, Old Slavonic": "cu",
 "Oromo": "om",
 "Oriya": "or",
 "Ossetian, Ossetic": "os",
 "Panjabi, Punjabi": "pa",
 "Pāli": "pi",
 "Persian": "fa",
 "Polish": "pl",
 "Pashto, Pushto": "ps",
 "Portuguese": "pt",
 "Quechua": "qu",
 "Romansh": "rm",
 "Kirundi": "rn",
 "Romanian, Moldavian, Moldovan": "ro",
 "Russian": "ru",
 "Sanskrit (Saṁskṛta)": "sa",
 "Sardinian": "sc",
 "Sindhi": "sd",
 "Northern Sami": "se",
 "Samoan": "sm",
 "Sango": "sg",
 "Serbian": "sr",
 "Scottish Gaelic; Gaelic": "gd",
 "Shona": "sn",
 "Sinhala, Sinhalese": "si",
 "Slovak": "sk",
 "Slovene": "sl",
 "Somali": "so",
 "Southern Sotho": "st",
 "Spanish; Castilian": "es",
 "Sundanese": "su",
 "Swahili": "sw",
 "Swati": "ss",
 "Swedish": "sv",
 "Tamil": "ta",
 "Telugu": "te",
 "Tajik": "tg",
 "Thai": "th",
 "Tigrinya": "ti",
 "Tibetan Standard, Tibetan, Central": "bo",
 "Turkmen": "tk",
 "Tagalog": "tl",
 "Tswana": "tn",
 "Tonga (Tonga Islands)": "to",
 "Turkish": "tr",
 "Tsonga": "ts",
 "Tatar": "tt",
 "Twi": "tw",
 "Tahitian": "ty",
 "Uighur, Uyghur": "ug",
 "Ukrainian": "uk",
 "Urdu": "ur",
 "Uzbek": "uz",
 "Venda": "ve",
 "Vietnamese": "vi",
 "Volapük": "vo",
 "Walloon": "wa",
 "Welsh": "cy",
 "Wolof": "wo",
 "Western Frisian": "fy",
 "Xhosa": "xh",
 "Yiddish": "yi",
 "Yoruba": "yo",
 "Zhuang, Chuang": "za"}'''
js = json.dumps(langs)

In [42]:
# translate
model = EasyNMT('opus-mt')
@app.get('/translate', response_class=HTMLResponse)
def translator(lang:str=''):
  prev_lang = 'en'
  if lang:
    for index in range(len(only_IT_jobs)):
      only_IT_jobs['job title'][index] = model.translate([only_IT_jobs.iloc[index]['job title']], target_lang = lang, source_lang = prev_lang)
      only_IT_jobs['Description'][index] = model.translate([only_IT_jobs.iloc[index]['Description']], target_lang = lang, source_lang = prev_lang)
      # if index != len(only_IT_jobs) - 1:
        # all_translated = f"""<html>
        #                   <h2><u>{index} / {len(only_IT_jobs)} Jobs translated <br> Try to choose other language maybe. </u></h2>
        #                   <p>
        #                   {js}
        #                   </p>
        #         </html>"""
        # return all_translated
      #   else:
    return Response(only_IT_jobs.to_json(orient = 'index'), media_type = 'application/json')
      # except Exception as e:
      #   error_message = f"""<html>
      #                       <p>
      #                       {e}
      #                       </p>
      #                       <h2><u>Oops! Something went wrong. <br> Try to choose other language maybe.</u></h2>
      #                       <p>
      #                       {js}
      #                       {e}
      #                       </p>
      #             </html>"""
      #   return error_message
  else:
    content = f"""<html>
                    <h2><u>List of all countries' languages code</u></h2>
                    <p>
                    {js}
                    </p>
                  </html>
              """ 
    return HTMLResponse(content)

In [ ]:
tunnel = ngrok.connect(8000)
print("PUBLIC URL: ", tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port = 8000)

PUBLIC URL:  http://e03d-34-142-169-164.ngrok.io


INFO:     Started server process [79]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     102.22.174.171:0 - "GET / HTTP/1.1" 200 OK
INFO:     102.22.174.171:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     102.22.174.171:0 - "GET /translate?lang=fr HTTP/1.1" 200 OK
INFO:     102.22.174.171:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
